In [ ]:
!pip install numpy==1.24.4 --upgrade --force-reinstall
!pip install faiss-cpu --no-cache-dir
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 38.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 328.2 MB/s eta 0:00:00


 Load and clean data

In [ ]:
import json
from urllib.parse import unquote


with open("products_raw.json", "r", encoding="utf-8") as f:
    data = json.load(f)

products = data["products"]

In [ ]:
clean_products = []

for raw in products :

  raw_id = raw.get("product_id", "")
  decoded_id = unquote(raw_id)  # (URL encoded)

  name = raw.get("product_name", "")
  category = raw.get("category_path", "")
  short_desc = raw.get("short_description", "")
  full_desc = raw.get("table_data", "")

  rag_text = f"""
اسم المنتج: {name}
التصنيف: {category}
الوصف:
{short_desc}
{full_desc}
""".strip()

  rag_text = rag_text.replace("\n", " ")

  clean_products.append({
    "product_id":decoded_id,
    "rag_text":rag_text


})
print(len(clean_products))
print(clean_products[0])

412
{'product_id': 'qarnabit-buni', 'rag_text': 'اسم المنتج: قرنبيط بوني التصنيف: مستلزمات الإنتاج النباتي > التقاوي والشتلات73 > تقاوي محاصيل الخضر58 > تقاوي القرنبيط1 الوصف: صنف قرنبيط ذو حجم رأس كبير يصلح للتصنيع و الأستهلاك الطازج وصف النبات: نبات قوي جدا يغطى القرص تماما وصف الثمار: الرأس ناصع البياض – الوزن من 5:4 كجم معدل التقاوي: بمعدل 10000 نبات للفدان صفات المقاومة للأمراض الآفات: متحمل للفيوزاريوم أهم المعاملات الزراعية للصنف: حقل مفتوح برنامج الري والتسميد للصنف: حسب نوع التربة ميعاد وطريقة الزراعة: سبتمبر الى ديسمبر ميعاد النضج: متوسط النضج 90 يوم من الزراعة الشركة المنتجة وبلد المنشأ: سينجينتا – هولنده أحجام العبوات المتوفرة: عبوة 2500 بيانات أخري عن الصنف: خريفي أسم المحصول: القنبيط مكان الزراعة: حقل مكشوف أنسب الاراضي للزراعة: ارض طميية , ارض رملية أنسب الاشهر للزراعة: سبتمبر, اكتوبر, نوفمبر, ديسمبر منشأ الصنف: مستورد أنسب المناطق للزراعة: الوجة البحري, الظهير الصحراوي للوجة البحري, الوجة القبلي, الظهير الصحراوي للوجة القبلي'}


 Encode using Sentence Transformer

In [ ]:
from sentence_transformers import SentenceTransformer



model_id="Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2"
device="cuda:0"
dim = 768

model=SentenceTransformer(model_id,device=device)

In [ ]:
all_texts = [prod["rag_text"] for prod in clean_products]

all_string_ids= [prod["product_id"] for prod in clean_products]

encoded_all_texts = model.encode(all_texts, convert_to_numpy=True,show_progress_bar=True)

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
len(encoded_all_texts)

412

 ID Mapping

In [ ]:
id_mapping = {s: i for i, s in enumerate(all_string_ids)}
reverse_mapping = {i: s for s, i in id_mapping.items()}  # للرجوع للـ string ID بعد البحث
all_int_ids = np.array([id_mapping[s] for s in all_string_ids], dtype='int64')

Build FAISS Index

In [ ]:
import faiss
import numpy as np

faiss.normalize_L2(encoded_all_texts)



In [ ]:
faiss_index = faiss.IndexIDMap(faiss.IndexFlatIP(dim))
faiss_index.add_with_ids(encoded_all_texts,all_int_ids )

In [ ]:
query = "أريد معلومات عن تقاوي القرنبيط"
embed_query=model.encode([query])

faiss.normalize_L2(embed_query)

result=faiss_index.search(embed_query,3)


result


(array([[0.69133043, 0.5697294 , 0.53278923]], dtype=float32),
 array([[ 0, 13,  1]]))

In [ ]:
all_texts[0]

'اسم المنتج: قرنبيط بوني التصنيف: مستلزمات الإنتاج النباتي > التقاوي والشتلات73 > تقاوي محاصيل الخضر58 > تقاوي القرنبيط1 الوصف: صنف قرنبيط ذو حجم رأس كبير يصلح للتصنيع و الأستهلاك الطازج وصف النبات: نبات قوي جدا يغطى القرص تماما وصف الثمار: الرأس ناصع البياض – الوزن من 5:4 كجم معدل التقاوي: بمعدل 10000 نبات للفدان صفات المقاومة للأمراض الآفات: متحمل للفيوزاريوم أهم المعاملات الزراعية للصنف: حقل مفتوح برنامج الري والتسميد للصنف: حسب نوع التربة ميعاد وطريقة الزراعة: سبتمبر الى ديسمبر ميعاد النضج: متوسط النضج 90 يوم من الزراعة الشركة المنتجة وبلد المنشأ: سينجينتا – هولنده أحجام العبوات المتوفرة: عبوة 2500 بيانات أخري عن الصنف: خريفي أسم المحصول: القنبيط مكان الزراعة: حقل مكشوف أنسب الاراضي للزراعة: ارض طميية , ارض رملية أنسب الاشهر للزراعة: سبتمبر, اكتوبر, نوفمبر, ديسمبر منشأ الصنف: مستورد أنسب المناطق للزراعة: الوجة البحري, الظهير الصحراوي للوجة البحري, الوجة القبلي, الظهير الصحراوي للوجة القبلي'

Prepare Metadata & Save

In [65]:
metadata = {}
for product in clean_products:
    product_id = product["product_id"]
    rag_text = product["rag_text"]

    # extract fields
    parts = rag_text.split("التصنيف:")
    name_part = parts[0].replace("اسم المنتج:", "").strip() if len(parts) > 0 else ""

    category_desc = parts[1].split("الوصف:") if len(parts) > 1 else ["", ""]
    category_part = category_desc[0].strip()

    metadata[product_id] = {
        "rag_text": rag_text,
        "product_name": name_part,
        "category": category_part
    }

# Save FAISS
faiss.write_index(faiss_index, "products.faiss")

# Save Metadata
with open("products_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

# Load test
index_loaded = faiss.read_index("products.faiss")

with open("products_metadata.json", "r", encoding="utf-8") as f:
    metadata_loaded = json.load(f)

print("\nتم التحميل بنجاح")



تم التحميل بنجاح
